# Word2Vector를 이용한 네이버 영화평 감정 분석

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Twitter
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
train_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_test.txt'

In [ ]:
with open( train_path, 'r', encoding = 'utf-8') as f:
  for line in f.read().splitlines():
    print("line =",line)
    print("line.split(\t)", line.split('\t'))
    print("="*100)

In [ ]:
# 테스트셋 로드
with open( train_path, 'r', encoding = 'utf-8') as f:

  result = [i.split('\t') for i in f.read().splitlines()]

# 첫 번쨰 칸 빼고 , 
result = result[1:]

train_tmp = result

In [ ]:
train_tmp

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1'],
 ['9008700', '걍인피니트가짱이다.진짜짱이다♥', '1'],
 ['10217543', '볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~', '1'],
 ['5957425', '울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해', '0'],
 ['8628627', '담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.', '1'],
 ['9864035', '취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지', '0'],
 ['6852435', 'ㄱ냥 매번 긴장되고 재밋음ㅠㅠ', '1'],
 ['9143163',
  '참 사람들 웃긴게 

In [ ]:
# 데스트셋 셋 로드
with open( test_path, 'r', encoding = 'utf-8') as f:

  result = [i.split('\t') for i in f.read().splitlines()]

# 첫 번쨰 칸 빼고 , 
result = result[1:]

test_tmp = result

In [ ]:
twitter = Twitter()

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
twitter.pos("안녕해", stem = True)

[('안녕하다', 'Adjective')]

In [ ]:
stems = [""] * 40

# 태깅 문장
text = "더이상 정복할 땅이 없자. 시저는 눈물을 흘렸다"

# 현재시제로 만든후 태깅
tagged = twitter.pos(text, stem=True)

stems_index = 0
for i in range(len(tagged)):
    # 태깅 결과
    print("tagged[{}]={}".format(i, tagged[i]))

    # 단어
    print("tagged[{}][0]={}".format(i, tagged[i][0]))

    # 품사
    print("tagged[{}][1]={}".format(i, tagged[i][1]))

    # 명사나 형용사면 추가
    if tagged[i][1]=="Noun" or tagged[i][1]=="Adjective":
        stems[stems_index] =  tagged[i][0]
        print("명사 형용사 라서 stems에 추가")
        stems_index +=1
    print("="*100)

tagged[0]=('더', 'Noun')
tagged[0][0]=더
tagged[0][1]=Noun
명사 형용사 라서 stems에 추가
tagged[1]=('이상', 'Noun')
tagged[1][0]=이상
tagged[1][1]=Noun
명사 형용사 라서 stems에 추가
tagged[2]=('정복', 'Noun')
tagged[2][0]=정복
tagged[2][1]=Noun
명사 형용사 라서 stems에 추가
tagged[3]=('하다', 'Verb')
tagged[3][0]=하다
tagged[3][1]=Verb
tagged[4]=('땅', 'Noun')
tagged[4][0]=땅
tagged[4][1]=Noun
명사 형용사 라서 stems에 추가
tagged[5]=('이', 'Josa')
tagged[5][0]=이
tagged[5][1]=Josa
tagged[6]=('없다', 'Adjective')
tagged[6][0]=없다
tagged[6][1]=Adjective
명사 형용사 라서 stems에 추가
tagged[7]=('.', 'Punctuation')
tagged[7][0]=.
tagged[7][1]=Punctuation
tagged[8]=('시저', 'Noun')
tagged[8][0]=시저
tagged[8][1]=Noun
명사 형용사 라서 stems에 추가
tagged[9]=('는', 'Josa')
tagged[9][0]=는
tagged[9][1]=Josa
tagged[10]=('눈물', 'Noun')
tagged[10][0]=눈물
tagged[10][1]=Noun
명사 형용사 라서 stems에 추가
tagged[11]=('을', 'Josa')
tagged[11][0]=을
tagged[11][1]=Josa
tagged[12]=('흘리다', 'Verb')
tagged[12][0]=흘리다
tagged[12][1]=Verb


In [ ]:
# 함수화
twitter = Twitter()

def getNounAdjective(text):
  # 단어 40개 추가할 리스트
  stems = [""] * 40

  # 인덱스 설정 , 40개 리스트의 마지막 인덱스는 39
  stems_last_index = (len(stems) - 1)

  stems_index = 0
  
  # 문장 품사 태깅
  tagged = twitter.pos(text, stem = True)

  for i in range(0, len(tagged)):
    # 명사나 형용사면 단어 추가
    if (tagged[i][1] == 'Noun') or (tagged[i][1] == 'Adjective'):
      stems[stems_index] = tagged[i][0]

      # 단어 40개 다 추가했으면 멈춤
      if stems_index == stems_last_index:
        break

      stems_index +=1

  # 명사나 형용사가 1개면 리턴 안받기
  if stems_index > 2:
    return stems

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
getNounAdjective("나는 달린다. 공양이가 날아온다 행복하다")

['나',
 '달린다',
 '공양',
 '행복하다',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [ ]:
for i in range(len(train_tmp[0:10])):
    print("train_tmp[{}]={}".format(i, train_tmp[i]) )
    print("train_tmp[{}][1]={}".format(i, train_tmp[i][1]) )
    print("train_tmp[{}][2]={}".format(i, train_tmp[i][2]) )
    print("getNounAdjective(train_tmp[i][1])=",getNounAdjective(train_tmp[i][1]))
    print("="*100)

train_tmp[0]=['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']
train_tmp[0][1]=아 더빙.. 진짜 짜증나네요 목소리
train_tmp[0][2]=0
getNounAdjective(train_tmp[i][1])= ['더빙', '진짜', '짜증나다', '목소리', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
train_tmp[1]=['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1']
train_tmp[1][1]=흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
train_tmp[1][2]=1
getNounAdjective(train_tmp[i][1])= ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
train_tmp[2]=['10265843', '너무재밓었다그래서보는것을추천한다', '0']
train_tmp[2][1]=너무재밓었다그래서보는것을추천한다
train_tmp[2][2]=0
getNounAdjective(train_tmp[i][1])= ['무재', '밓었', '다그', '래서', '추천', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
train_tmp[3]=['9045019', '교

In [ ]:
def kor_movie():

  # 학습 데이터로 사용할 리뷰데이터 저장 리스트
  train_x = []

  # 정답 데이터로 사용할 0,1 을 저장 리스트
  train_y = []
  for i in tqdm(range(len(train_tmp))):
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(train_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      train_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      train_y.append(int(train_tmp[i][2]))

  # 검증 데이터로 사용할 리뷰데이터 저장 리스트
  test_x = []

  # 검증 데이터의 정답 리스트
  test_y = []
  for i in tqdm(range(len(test_tmp))):
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(test_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      test_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      test_y.append(int(train_tmp[i][2]))
  return (train_x, train_y), (test_x, test_y)

In [ ]:
(X_train, y_train) , (X_test, y_test) = kor_movie()

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
X_train

In [ ]:
word2vector = Word2Vec(X_train, size = 20)

In [ ]:
word2vector.wv.vocab

{'더빙': <gensim.models.keyedvectors.Vocab at 0x7fcc8babb8d0>,
 '진짜': <gensim.models.keyedvectors.Vocab at 0x7fccbe742a90>,
 '짜증나다': <gensim.models.keyedvectors.Vocab at 0x7fccbe742d90>,
 '목소리': <gensim.models.keyedvectors.Vocab at 0x7fccbe742890>,
 '': <gensim.models.keyedvectors.Vocab at 0x7fcc91dca990>,
 '흠': <gensim.models.keyedvectors.Vocab at 0x7fcc9968c1d0>,
 '포스터': <gensim.models.keyedvectors.Vocab at 0x7fcc9968cb90>,
 '보고': <gensim.models.keyedvectors.Vocab at 0x7fcc9968c9d0>,
 '초딩': <gensim.models.keyedvectors.Vocab at 0x7fcc91dcacd0>,
 '영화': <gensim.models.keyedvectors.Vocab at 0x7fcc92574850>,
 '줄': <gensim.models.keyedvectors.Vocab at 0x7fcc8ba51ed0>,
 '오버': <gensim.models.keyedvectors.Vocab at 0x7fcc8ba51c10>,
 '연기': <gensim.models.keyedvectors.Vocab at 0x7fcc8ba51f10>,
 '가볍다': <gensim.models.keyedvectors.Vocab at 0x7fcc8ba51e90>,
 '무재': <gensim.models.keyedvectors.Vocab at 0x7fcc96f3edd0>,
 '다그': <gensim.models.keyedvectors.Vocab at 0x7fcc96f3ef90>,
 '래서': <gensim.models.k

In [ ]:
word_vector_keys = word2vector.wv.vocab.keys()
word_vector_keys

dict_keys(['더빙', '진짜', '짜증나다', '목소리', '', '흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '무재', '다그', '래서', '추천', '교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정', '의', '익살스럽다', '스파이더맨', '이쁘다', '막', '걸음', '마', '세', '초등학교', '학년', '생인', '반개', '아깝다', '움', '원작', '긴장감', '제대로', '별', '욕', '이응경', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '드라마', '가족', '사람', '네', '액션', '있다', '몇', '안되다', '왜케', '낮다', '꽤', '볼', '데', '헐리우드', '화려하다', '인피니트', '짱', '볼때', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인', '울면', '손', '때', '뻔', '이범수', '드럽다', '담백하다', '깔끔하다', '좋다', '로만', '자꾸', '그', '것', '취향', '존중', '다지', '내생', '극장', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', '냥', '매번', '긴장', '재밋음', '바스코', '이기', '락스', '코', '바비', '아이돌', '그냥', '안달', '굿바이', '레닌', '표절', '이해', '왜', '뒤', '갈수록', '이건', '깨알', '캐스팅', '산뜻하다', '용구성', '일드', '위', '변명', '놈', '착하다', '절대', '아니다', '걸', '나름', '심오하다', '뜻', '듯', '학생', '선생', '재미없다', '지루하다', '같다', '음식', '도', '만찬', '별로', '핀란드', '풍경', '평범하다', '수작', '는걸', '주제', '중반', '납득', '수', '그렇다', '꼭', '

In [ ]:
# 영화와 비슷한 단어와 유사도
word2vector.wv.most_similar("영화")

[('영화로', 0.889551043510437),
 ('애니메이션', 0.7982966899871826),
 ('재난영화', 0.7862777709960938),
 ('전쟁영화', 0.7797781229019165),
 ('일본사람', 0.7719219326972961),
 ('롤링', 0.7713843584060669),
 ('액션영화', 0.7632489800453186),
 ('다큐', 0.7630776166915894),
 ('비디오영화', 0.7614374756813049),
 ('연화', 0.7571612000465393)]

In [ ]:
# 배우와 비슷한 단어와 유사도
word2vector.wv.most_similar("배우")

[('연기자', 0.9448115825653076),
 ('조연', 0.8836774826049805),
 ('배역', 0.8646745085716248),
 ('진과', 0.8491079211235046),
 ('외모', 0.8277369737625122),
 ('주연', 0.8185327649116516),
 ('개발', 0.8177402019500732),
 ('송강호', 0.8169464468955994),
 ('신인', 0.8157616853713989),
 ('배우진', 0.8092398643493652)]

In [ ]:
# 영화의 벡터값
word2vector['영화']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([ 2.9870124e+00, -5.0440139e-01,  3.6639923e-01,  1.4068604e+00,
       -2.4765465e+00,  1.1199456e+00,  4.9149400e-01, -1.1885792e-03,
       -3.1621959e+00,  4.2544193e+00,  1.6605721e-01, -7.9890746e-01,
       -7.2618979e-01,  8.6117029e-01, -4.9819517e+00, -2.9227027e-01,
       -2.8989666e+00, -2.0066266e-01,  1.7274566e+00, -1.5639291e+00],
      dtype=float32)

In [ ]:
print("영화" in word_vector_keys)

# 빈도수가 최소 5여야하는데 아닌 경우는 벡터값 안찾아줌
print("밓었" in word_vector_keys)

True
False


In [ ]:
# 학습할때 "영화"라는 단어를 학습 못함 따라서 위와 같은 벡터값으로 변경
for index in range(3):
    print("X_train[{}]={}".format(index, X_train[index]))
    for word in X_train[index]:
        print("word=",word)
        if word in word_vector_keys:
            print("word2vector[{}]={}".format(word, word2vector[word]))
        else:
            print("word2vector['']={}".format(word2vector['']))

        print("*"*100)
        
    print("="*100)

X_train[0]=['더빙', '진짜', '짜증나다', '목소리', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
word= 더빙
word2vector[더빙]=[-0.9693797   1.3657986   3.2104177  -1.3626442  -0.87579054  2.490588
 -0.89543176 -3.0370746  -1.9731306   1.1572696   0.69912    -2.5723429
  1.0157431  -1.3525244  -2.4263432  -2.326604    2.7599125  -0.33703858
  1.8072277  -0.59209013]
****************************************************************************************************
word= 진짜
word2vector[진짜]=[ 2.8361839e-01 -2.4064088e-01  1.3616796e+00 -3.9816496e-01
 -1.7294580e+00  4.1059456e+00 -1.9039174e+00 -2.9861531e+00
 -2.3257107e-01 -7.2961114e-04 -1.5945076e+00 -3.1590500e+00
  1.0896116e+00 -1.5457357e+00 -4.7104836e+00  2.9972785e+00
  1.2170788e-01 -4.3172574e-01  2.1163130e+00  2.7623949e+00]
****************************************************************************************************
word= 짜증나다
word2v

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


****************************************************************************************************
word= 
word2vector[]=[ 0.7995605   0.13340706 -0.1807313  -0.93127596  1.1055499   2.0073955
  0.25752684  0.42016193 -1.2064086   0.57442605  0.18000641 -0.21277471
 -0.00219649 -1.1856656  -1.6287998   1.0619525   0.23512098 -0.28200266
  1.1603652   1.8240999 ]
****************************************************************************************************
word= 
word2vector[]=[ 0.7995605   0.13340706 -0.1807313  -0.93127596  1.1055499   2.0073955
  0.25752684  0.42016193 -1.2064086   0.57442605  0.18000641 -0.21277471
 -0.00219649 -1.1856656  -1.6287998   1.0619525   0.23512098 -0.28200266
  1.1603652   1.8240999 ]
****************************************************************************************************
word= 
word2vector[]=[ 0.7995605   0.13340706 -0.1807313  -0.93127596  1.1055499   2.0073955
  0.25752684  0.42016193 -1.2064086   0.57442605  0.18000641 -0.21277471
 

In [ ]:
X_train_vector=[]
for index in tqdm(range(len(X_train))):
    input =[]

    for word in X_train[index]:
        # print("word = ", word)
        if word in word_vector_keys:
            input.append(word2vector[word].tolist())
        else:
            input.append(word2vector[""].tolist())
    
    # print("input=",input)
    X_train_vector.append(input)

  0%|          | 0/125008 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# 타입 변경
X_train_vector = np.array(X_train_vector, dtype="float32")

In [ ]:
X_train_vector[0].shape

(40, 20)

In [ ]:
# 모델 
model = Sequential()

# input_shape (단어 갯수(40단어) , 단어 당 칸수(20칸, 단어 벡터))
model.add(LSTM(units = 128, input_shape=(40, 20)))

#
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               76288     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 76,417
Trainable params: 76,417
Non-trainable params: 0
_________________________________________________________________


In [ ]:
y_train = np.array(y_train, dtype = np.float32)

In [ ]:
model.compile(
               loss="binary_crossentropy",
               optimizer=Adam(lr=1e-3),
               metrics=["acc"]
              )
model.fit(X_train_vector, y_train, batch_size =1000, epochs = 100 )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
126/126 [==============================] - 73s 547ms/step - loss: 0.5203 - acc: 0.7317
Epoch 2/100
 81/126 [==================>...........] - ETA: 24s - loss: 0.4617 - acc: 0.7856

# 실습

## WordVector를 이용한 네이버 영화평 감정 분석
- 194~218

단계
1. 데이터 로드 : read_data() 함수
2. 태깅 품사 중 명사랑 형용사만 리스트에 추가 하는 함수 : getNounAdjective
3. 전체 데이터 40개씩 품사 태깅하고 (X_train)리스트에 추가
4. X_train을 벡터로 변환하기위해 빈도수 체크하고 디폴트값(5) 5번 이상 나온것들만 리스트에 추가, 5이하인것들은 공백으로 처리
5. 모델 학습

In [ ]:
!pip install konlpy

In [3]:
from konlpy.tag import Twitter
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [4]:
train_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_train.txt'
test_path = '/content/drive/MyDrive/Colab Notebooks/AI_허현준/15주차 RNN_LSTM/datasets/ratings_test.txt'

In [5]:
def read_data(filename):
  with open(filename, 'r', encoding = 'utf-8') as f:
    # 한문장씩 끊어서 탭 기준으로 분리해서 리스트에 추가
    result = [line.split('\t') for line in f.read().splitlines()]

    # 칸 이름 뺴고 저장
    result = result[1:]
  return result

In [6]:
train_tmp = read_data(train_path)
test_tmp = read_data(test_path)

In [7]:
train_tmp[0:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

In [8]:
# 함수화
twitter = Twitter()

def getNounAdjective(text):
  # 단어 40개 추가할 리스트
  stems = [""] * 40

  # 인덱스 설정 , 40개 리스트의 마지막 인덱스는 39
  stems_last_index = (len(stems) - 1)

  stems_index = 0
  
  # 문장 품사 태깅
  tagged = twitter.pos(text, stem = True)

  for i in range(0, len(tagged)):
    # 명사나 형용사면 단어 추가
    if (tagged[i][1] == 'Noun') or (tagged[i][1] == 'Adjective'):
      stems[stems_index] = tagged[i][0]
      stems_index +=1

    # 단어 40개 다 추가했으면 멈춤
    if stems_index == stems_last_index:
        break

  # 명사나 형용사가 1개면 리턴 안받기
  if stems_index > 2:
    return stems

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [9]:
getNounAdjective(train_tmp[0][1])

['더빙',
 '진짜',
 '짜증나다',
 '목소리',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [10]:
train_tmp[1][1]

'흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [11]:
getNounAdjective(train_tmp[1][1])

['흠',
 '포스터',
 '보고',
 '초딩',
 '영화',
 '줄',
 '오버',
 '연기',
 '가볍다',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [12]:
def kor_movie():

  # 학습 데이터로 사용할 리뷰데이터 저장 리스트
  train_x = []

  # 정답 데이터로 사용할 0,1 을 저장 리스트
  train_y = []
  for i in tqdm(range(len(train_tmp))):
    
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(train_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      train_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      train_y.append(int(train_tmp[i][2]))

  # 검증 데이터로 사용할 리뷰데이터 저장 리스트
  test_x = []

  # 검증 데이터의 정답 리스트
  test_y = []
  for i in tqdm(range(len(test_tmp))):
    # getNounAdjective 함수 써서 명사랑 형용사만 리턴
    noun_adjective = getNounAdjective(test_tmp[i][1])

    # None값이 아니면
    if noun_adjective is not None:

      # 리스트에 추가
      test_x.append(noun_adjective)

      # 0,1 ,값을 정수로 변환해서 저장
      test_y.append(int(train_tmp[i][2]))
  return (train_x, train_y), (test_x, test_y)

In [13]:
(X_train, y_train) , (X_test, y_test) = kor_movie()

  0%|          | 0/150000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [14]:
X_train[1]

['흠',
 '포스터',
 '보고',
 '초딩',
 '영화',
 '줄',
 '오버',
 '연기',
 '가볍다',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [15]:
del train_tmp
del test_tmp

In [16]:
# X_train 단어를 워드 벡터로 변환
# size = 20 , 20 개 칸수
word2vector = Word2Vec(X_train, size =20)

# 워드벡터 계산할때 사용한 변수들을 메모리에서삭제
word2vector.init_sims(replace = True)

In [17]:
# word2vector에 저장된 단어들을 딕셔너리 객체로 리턴후 키값 리턴
word_vector_keys =  word2vector.wv.vocab.keys()
word_vector_keys

dict_keys(['더빙', '진짜', '짜증나다', '목소리', '', '흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '가볍다', '무재', '다그', '래서', '추천', '교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정', '의', '익살스럽다', '스파이더맨', '이쁘다', '막', '걸음', '마', '세', '초등학교', '학년', '생인', '반개', '아깝다', '움', '원작', '긴장감', '제대로', '별', '욕', '이응경', '생활', '년', '정말', '발', '해도', '그것', '납치', '감금', '반복', '드라마', '가족', '사람', '네', '액션', '있다', '몇', '안되다', '왜케', '낮다', '꽤', '볼', '데', '헐리우드', '화려하다', '인피니트', '짱', '볼때', '향수', '자극', '허진호', '감성', '절제', '멜로', '달인', '울면', '손', '때', '뻔', '이범수', '드럽다', '담백하다', '깔끔하다', '좋다', '로만', '자꾸', '그', '것', '취향', '존중', '다지', '내생', '극장', '가장', '노잼', '노', '감동', '임', '스토리', '어거지', '냥', '매번', '긴장', '재밋음', '바스코', '이기', '락스', '코', '바비', '아이돌', '그냥', '안달', '굿바이', '레닌', '표절', '이해', '왜', '뒤', '갈수록', '이건', '깨알', '캐스팅', '산뜻하다', '용구성', '일드', '위', '변명', '놈', '착하다', '절대', '아니다', '걸', '나름', '심오하다', '뜻', '듯', '학생', '선생', '재미없다', '지루하다', '같다', '음식', '도', '만찬', '별로', '핀란드', '풍경', '평범하다', '수작', '는걸', '주제', '중반', '납득', '수', '그렇다', '꼭', '

In [18]:
# X_train 문자열을 word2vector로 변환해서 저장할 배열
X_train_vector = []

for index in tqdm(range(len(X_train))):
    input = []

    for word in X_train[index]:
        # X_train의 단어가 word_vector_keys에 있으면 추가
        # 없는 경우는 빈도수가 낮은 경우 없음
        if word in word_vector_keys:
         input.append(word2vector[word].tolist())
        else:
         input.append(word2vector[""].tolist())
    
    X_train_vector.append(input)

# 타입 변환
X_train_vector = np.array(X_train_vector, dtype = np.float32)

# 삭제
del X_train

  0%|          | 0/125008 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [19]:
# X_train 문자열을 word2vector로 변환해서 저장할 배열
X_test_vector = []

for index in tqdm(range(len(X_test))):
    input = []

    for word in X_test[index]:
        # X_train의 단어가 word_vector_keys에 있으면 추가
        # 없는 경우는 빈도수가 낮은 경우 없음
        if word in word_vector_keys:
         input.append(word2vector[word].tolist())
        else:
         input.append(word2vector[""].tolist())
    
    X_test_vector.append(input)

# 타입 변환
X_test_vector = np.array(X_test_vector, dtype = np.float32)

# 삭제
del X_test

  0%|          | 0/41727 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [20]:
# 모델 
model = Sequential()

# input_shape (단어 갯수(40단어) , 단어 당 칸수(20칸, 단어 벡터))
model.add(LSTM(units = 128, input_shape=(40, 20)))

#
model.add(Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               76288     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 76,417
Trainable params: 76,417
Non-trainable params: 0
_________________________________________________________________


In [21]:
y_train = np.array(y_train, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)

In [22]:
model.compile(
               loss="binary_crossentropy",
               optimizer=Adam(lr=1e-3),
               metrics=["acc"]
              )
model.fit(X_train_vector, y_train, batch_size =1000, epochs = 100 )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
126/126 [==============================] - 13s 34ms/step - loss: 0.5420 - acc: 0.6970
Epoch 2/100
126/126 [==============================] - 5s 33ms/step - loss: 0.4439 - acc: 0.7947
Epoch 3/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4424 - acc: 0.7953
Epoch 4/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4411 - acc: 0.7946
Epoch 5/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4402 - acc: 0.7962
Epoch 6/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4355 - acc: 0.7984
Epoch 7/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4294 - acc: 0.8013
Epoch 8/100
126/126 [==============================] - 4s 34ms/step - loss: 0.4284 - acc: 0.8007
Epoch 9/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4296 - acc: 0.8009
Epoch 10/100
126/126 [==============================] - 4s 33ms/step - loss: 0.4211 - acc: 0.8050
Epoch 11/100
126/126 [======

In [23]:
model.evaluate(X_test_vector, y_test)

1304/1304 [==============================] - 8s 5ms/step - loss: 1.3604 - acc: 0.4979


[1.360398530960083, 0.4978790581226349]